In [1]:
import pandas as pd
import plotly.express as px
import pycountry
import pycountry_convert
import plotly.express as px
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

In [2]:

decision = pd.read_csv('../data/asylum-decisions.csv')  

In [3]:
#convert ISO country code to continent name
def iso_to_continent_name(iso_code):
    if isinstance(iso_code, str) and len(iso_code) == 3:
        try:
            alpha2_code = pycountry_convert.country_alpha3_to_country_alpha2(iso_code)
            continent_code = pycountry_convert.country_alpha2_to_continent_code(alpha2_code)
            continent_name = pycountry_convert.convert_continent_code_to_continent_name(continent_code)
            return continent_name
        except KeyError:
            return "Unknown"
    else:
        return "Unknown"

#apply the function to the "Country of origin (ISO)" column
decision["Continent"] = decision["Country of origin (ISO)"].apply(iso_to_continent_name)

In [4]:
# remove rows with 0 total decisions
decision1 = decision[decision['Total decisions'] != 0]
decision1 = decision1[decision1['Continent'] != "Unknown"]


In [5]:
# calculate the approval rate
decision['Approval rate'] = 1 - (decision['Rejected decisions'] / decision['Total decisions'])

decision1 = decision[decision['Total decisions'] != 0]
decision1 = decision1[decision1['Continent'] != "Unknown"]

d1 = decision1.groupby(["Continent", "Year"]).mean().reset_index()
d1 = d1[d1["Approval rate"] != 0]
d1 = d1[d1["Approval rate"] != 1]
fig = px.box(d1, x='Continent', y='Approval rate')

fig.update_layout(
    title=dict(
        text='Average Approval Rate of Asylum Application by Continent',
        font=dict(
            family="Arial",
            size=20
        )
    ),
    xaxis_title=dict(
        text='Continent',
        font=dict(
            family="Arial",
            size=18
        )
    ),
    yaxis_title=dict(
        text='Approval rate',
        font=dict(
            family="Arial",
            size=18
        )
    ),
    width=600,
    height=600,
    template="plotly_white"
)
fig.update_xaxes(tickangle=45)
fig.show()